# Initialization

Test notebook for the DataHandlers. Test the CMAPSS DataHandler

First we import the necessary packages and create the global variables.

In [1]:
import os
import numpy as np
import math
import tensorflow as tf

from time import time
from datetime import datetime
from keras.callbacks import TensorBoard

from data_handler_CMAPS import CMAPSDataHandler
from tunable_model import SequenceTunableModelRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D, LSTM
from keras.optimizers import Adam, SGD
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from keras import regularizers

import CMAPSAuxFunctions
from CMAPSAuxFunctions import TrainValTensorBoard

from numpy.random import seed
from tensorflow import set_random_seed


Using TensorFlow backend.


# Define different types of Keras models

In [2]:
l2_lambda_regularization = 0.2

def RULmodel_SN_5(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(12, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), 
                    name='fc1'))
    model.add(Dense(16, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), 
                    name='fc2'))
    model.add(Dense(12, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), 
                    name='fc3'))
    model.add(Dense(1, activation='linear', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(0.1), name='out'))
    
    return model

def test_model(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(20, input_dim=input_shape, activation='relu', 
                    kernel_initializer=keras.initializers.glorot_normal(seed=0), 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), name='fc1'))
    model.add(Dense(1, activation='linear', 
                    kernel_initializer=keras.initializers.glorot_normal(seed=0), 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), name='out'))
    
    return model

# Create the Data Handler

In [3]:
#Selected as per CNN paper
features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
selected_features = list(features[i] for i in selected_indices-1)
data_folder = '../CMAPSSData'

window_size = 30
window_stride = 1
max_rul = 125

dHandler_cmaps = CMAPSDataHandler(data_folder, 1, selected_features, max_rul, window_size, window_stride)
#dHandler_cmaps.load_data(verbose=1, cross_validation_ratio=0.2)

# Create Tunable Model

In [4]:
min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
#min_max_scaler = None

In [5]:
def get_compiled_model(shape, model_type='ann'):
    
    K.clear_session()  #Clear the previous tensorflow graph
    
    #To test the model without randomness

    
    #Shared parameters for the models
    optimizer = Adam(lr=0,beta_1=0.5)
    
    lossFunction = "mean_squared_error"
    metrics = ["mse"]
    model = None

    #Create and compile the models
    model = test_model(shape)
    model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

    return model




In [6]:
num_features = len(selected_features)

shape = len(selected_features)*window_size
model = get_compiled_model(shape, model_type='ann')
tModel = SequenceTunableModelRegression('ModelRUL_SN_1', model, lib_type='keras', data_handler=dHandler_cmaps)


# Load Data

In [7]:
tModel.data_handler.sequence_length = 30
#tModel.data_handler.sequence_length = maxWindowSize[datasetNumber]
tModel.data_handler.sequence_stride = 1
tModel.data_handler.max_rul = 129

In [8]:
#Load Non sequenced data (unroll sequence into a single feature vector)

tModel.data_handler.data_scaler = None
tModel.data_scaler = min_max_scaler

tModel.load_data(unroll=True, verbose=1, cross_validation_ratio=0.2)
#tModel.data_handler.print_data()
tModel.print_data()

Loading data for dataset 1 with window_size of 30, stride of 1 and maxRUL of 129. Cros-Validation ratio 0.2
Loading data from file and computing dataframes
Printing shapes

Training data (X, y)
(13807, 420)
(13807, 1)
Cross-Validation data (X, y)
(20, 420)
(20, 1)
Testing data (X, y)
(100, 420)
(100, 1)
Printing first 5 elements

Training data (X, y)
[[-0.58075601 -0.0455243  -0.25933401 ... -0.8         0.43307087
   0.45804398]
 [-0.35395189  0.0629156  -0.1568113  ... -0.4         0.25984252
   0.23900463]
 [-0.21649485 -0.13299233 -0.11402624 ... -0.4         0.38582677
   0.71527778]
 [-0.21649485 -0.39897698 -0.20807265 ... -0.4         0.08661417
   0.28327546]
 [-0.20274914 -0.39590793 -0.03249243 ... -0.4         0.05511811
   0.1634838 ]]
[[129.]
 [129.]
 [129.]
 [129.]
 [129.]]
Cross-Validation data (X, y)
[[ 0.395189    0.04910486  0.12734612 ...  0.          0.02362205
  -0.0552662 ]
 [ 0.21649485 -0.09820972 -0.09021191 ...  0.         -0.02362205
  -0.21614583]
 [ 0.1752

# Train the model and test some Tunable Model functionalities

In [9]:
nFeatures = len(selected_features)

lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)

#earlyStopping=keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, verbose=0, mode='auto')

shape = (nFeatures*tModel.data_handler.sequence_length)
print(shape)


model = get_compiled_model(shape, model_type='ann')
tModel.change_model('ModelRUL_SN_1', model, 'keras')

now = datetime.now()
tensorboard = TrainValTensorBoard(log_dir="./logs/"+now.strftime("%Y%m%d-%H%M%S")+ "/",write_graph=False)
#,tensorboard = tensorboard
tModel.epochs = 200
tModel.train_model(learningRate_scheduler=lrate, verbose=0,tensorboard = tensorboard)




420
training with cv
Logs: dict_items([('val_loss', 9459.365234375), ('val_mean_squared_error', 9450.8291015625), ('loss', 7912.218541135724), ('mean_squared_error', 7904.078690623982), ('lr', 0.001)])
Val_logs: {'loss': 9459.365234375, 'mean_squared_error': 9450.8291015625}
Logs: dict_items([('val_loss', 8312.857421875), ('val_mean_squared_error', 8302.3798828125), ('loss', 6746.4579150507325), ('mean_squared_error', 6737.08915519811), ('lr', 0.001)])
Val_logs: {'loss': 8312.857421875, 'mean_squared_error': 8302.3798828125}
Logs: dict_items([('val_loss', 6957.4345703125), ('val_mean_squared_error', 6943.72265625), ('loss', 5448.09936698493), ('mean_squared_error', 5436.202556511473), ('lr', 0.001)])
Val_logs: {'loss': 6957.4345703125, 'mean_squared_error': 6943.72265625}
Logs: dict_items([('val_loss', 5588.2900390625), ('val_mean_squared_error', 5569.7568359375), ('loss', 4083.193305355328), ('mean_squared_error', 4067.2346028021057), ('lr', 0.001)])
Val_logs: {'loss': 5588.2900390625

Logs: dict_items([('val_loss', 978.5529174804688), ('val_mean_squared_error', 910.7841796875), ('loss', 343.63839693829937), ('mean_squared_error', 275.94203164263723), ('lr', 0.001)])
Val_logs: {'loss': 978.5529174804688, 'mean_squared_error': 910.7841796875}
Logs: dict_items([('val_loss', 970.5819091796875), ('val_mean_squared_error', 902.74658203125), ('loss', 341.57310657027386), ('mean_squared_error', 273.7408768001901), ('lr', 0.001)])
Val_logs: {'loss': 970.5819091796875, 'mean_squared_error': 902.74658203125}
Logs: dict_items([('val_loss', 970.4615478515625), ('val_mean_squared_error', 902.5989379882812), ('loss', 339.70010175281544), ('mean_squared_error', 271.87412011370407), ('lr', 0.001)])
Val_logs: {'loss': 970.4615478515625, 'mean_squared_error': 902.5989379882812}
Logs: dict_items([('val_loss', 972.4848022460938), ('val_mean_squared_error', 904.6788330078125), ('loss', 338.09105273191716), ('mean_squared_error', 270.23280628331713), ('lr', 0.001)])
Val_logs: {'loss': 972

Logs: dict_items([('val_loss', 931.923828125), ('val_mean_squared_error', 870.4091796875), ('loss', 305.1112489035156), ('mean_squared_error', 243.49176212823858), ('lr', 0.001)])
Val_logs: {'loss': 931.923828125, 'mean_squared_error': 870.4091796875}
Logs: dict_items([('val_loss', 945.9542236328125), ('val_mean_squared_error', 884.6387939453125), ('loss', 304.2406339636406), ('mean_squared_error', 242.82059815553853), ('lr', 0.001)])
Val_logs: {'loss': 945.9542236328125, 'mean_squared_error': 884.6387939453125}
Logs: dict_items([('val_loss', 943.4249267578125), ('val_mean_squared_error', 882.3023681640625), ('loss', 303.4150537035109), ('mean_squared_error', 242.23386919219567), ('lr', 0.001)])
Val_logs: {'loss': 943.4249267578125, 'mean_squared_error': 882.3023681640625}
Logs: dict_items([('val_loss', 944.60791015625), ('val_mean_squared_error', 883.70068359375), ('loss', 303.0140047546506), ('mean_squared_error', 241.9931324441937), ('lr', 0.001)])
Val_logs: {'loss': 944.60791015625

Logs: dict_items([('val_loss', 919.9805908203125), ('val_mean_squared_error', 864.1222534179688), ('loss', 287.126862310505), ('mean_squared_error', 231.26880249390348), ('lr', 1e-04)])
Val_logs: {'loss': 919.9805908203125, 'mean_squared_error': 864.1222534179688}
Logs: dict_items([('val_loss', 920.9467163085938), ('val_mean_squared_error', 865.1060791015625), ('loss', 287.10619527480117), ('mean_squared_error', 231.26026158261337), ('lr', 1e-04)])
Val_logs: {'loss': 920.9467163085938, 'mean_squared_error': 865.1060791015625}
Logs: dict_items([('val_loss', 924.021240234375), ('val_mean_squared_error', 868.19873046875), ('loss', 287.0741249560239), ('mean_squared_error', 231.2439908839726), ('lr', 1e-04)])
Val_logs: {'loss': 924.021240234375, 'mean_squared_error': 868.19873046875}
Logs: dict_items([('val_loss', 921.0638427734375), ('val_mean_squared_error', 865.2576293945312), ('loss', 286.98801457981216), ('mean_squared_error', 231.17442514183745), ('lr', 1e-04)])
Val_logs: {'loss': 92

Logs: dict_items([('val_loss', 922.2219848632812), ('val_mean_squared_error', 867.1018676757812), ('loss', 285.13304527306474), ('mean_squared_error', 229.99754117992083), ('lr', 1e-04)])
Val_logs: {'loss': 922.2219848632812, 'mean_squared_error': 867.1018676757812}
Logs: dict_items([('val_loss', 926.0681762695312), ('val_mean_squared_error', 870.9803466796875), ('loss', 285.03839085025584), ('mean_squared_error', 229.92852036122557), ('lr', 1e-04)])
Val_logs: {'loss': 926.0681762695312, 'mean_squared_error': 870.9803466796875}
Logs: dict_items([('val_loss', 913.7139892578125), ('val_mean_squared_error', 858.6300048828125), ('loss', 284.98420039533556), ('mean_squared_error', 229.9101384117937), ('lr', 1e-04)])
Val_logs: {'loss': 913.7139892578125, 'mean_squared_error': 858.6300048828125}
Logs: dict_items([('val_loss', 924.2277221679688), ('val_mean_squared_error', 869.1805419921875), ('loss', 284.9881309939733), ('mean_squared_error', 229.9309476259919), ('lr', 1e-04)])
Val_logs: {'lo

Logs: dict_items([('val_loss', 922.3161010742188), ('val_mean_squared_error', 868.1478271484375), ('loss', 282.57922465396996), ('mean_squared_error', 228.38287286212065), ('lr', 1e-04)])
Val_logs: {'loss': 922.3161010742188, 'mean_squared_error': 868.1478271484375}
Logs: dict_items([('val_loss', 916.4476318359375), ('val_mean_squared_error', 862.2911376953125), ('loss', 282.5405448294138), ('mean_squared_error', 228.385325763023), ('lr', 1e-04)])
Val_logs: {'loss': 916.4476318359375, 'mean_squared_error': 862.2911376953125}
Logs: dict_items([('val_loss', 920.9664916992188), ('val_mean_squared_error', 866.8546752929688), ('loss', 282.43289526724374), ('mean_squared_error', 228.29009794040584), ('lr', 1e-04)])
Val_logs: {'loss': 920.9664916992188, 'mean_squared_error': 866.8546752929688}
Logs: dict_items([('val_loss', 918.3815307617188), ('val_mean_squared_error', 864.2940673828125), ('loss', 282.3513091446446), ('mean_squared_error', 228.26051302495628), ('lr', 1e-04)])
Val_logs: {'los

In [10]:
tModel.evaluate_model(['rhs', 'rmse'], round=2, cross_validation=True)
print("scores")

cScores = tModel.scores
#rmse = math.sqrt(cScores['score_1'])
rmse2 = cScores['rmse']
rhs = cScores['rhs']
time = tModel.train_time

i = range(1,len(tModel.y_test)+1)


for i, rul_prediction, rul_prediction_rounded, true_rul in zip(i, np.ravel(tModel.y_predicted), tModel.y_predicted_rounded, np.ravel(tModel.y_test)):
    print('Engine {}, Predicted RUL {}, Rounded RUL {}, Real RUL {}'.format(i, rul_prediction, 
                                                                    rul_prediction_rounded, 
                                                                    true_rul))

print(cScores)
#print("RMSE: {}".format(rmse))
print("RMSE2: {}".format(rmse2))
print("RHS: {}".format(rhs))
print("Time : {} seconds".format(time))

20/20 [==============================] - 0s 32us/step
scores
Engine 1, Predicted RUL 60.17877197265625, Rounded RUL 60.0, Real RUL 112.0
Engine 2, Predicted RUL 34.336456298828125, Rounded RUL 34.0, Real RUL 98.0
Engine 3, Predicted RUL 79.47891235351562, Rounded RUL 79.0, Real RUL 69.0
Engine 4, Predicted RUL 25.719270706176758, Rounded RUL 25.0, Real RUL 82.0
Engine 5, Predicted RUL 95.18194580078125, Rounded RUL 95.0, Real RUL 91.0
Engine 6, Predicted RUL 117.83250427246094, Rounded RUL 117.0, Real RUL 93.0
Engine 7, Predicted RUL 117.86837768554688, Rounded RUL 117.0, Real RUL 91.0
Engine 8, Predicted RUL 42.33420944213867, Rounded RUL 42.0, Real RUL 95.0
Engine 9, Predicted RUL 98.05426788330078, Rounded RUL 98.0, Real RUL 111.0
Engine 10, Predicted RUL 89.20503997802734, Rounded RUL 89.0, Real RUL 96.0
Engine 11, Predicted RUL 84.1662826538086, Rounded RUL 84.0, Real RUL 97.0
Engine 12, Predicted RUL 110.2314453125, Rounded RUL 110.0, Real RUL 124.0
Engine 13, Predicted RUL 124.3

In [11]:
tModel.evaluate_model(['rhs', 'rmse'], round=2, cross_validation=False)
print("scores")

cScores = tModel.scores
#rmse = math.sqrt(cScores['score_1'])
rmse2 = cScores['rmse']
rhs = cScores['rhs']
time = tModel.train_time

i = range(1,len(tModel.y_test)+1)


for i, rul_prediction, rul_prediction_rounded, true_rul in zip(i, np.ravel(tModel.y_predicted), tModel.y_predicted_rounded, np.ravel(tModel.y_test)):
    print('Engine {}, Predicted RUL {}, Rounded RUL {}, Real RUL {}'.format(i, rul_prediction, 
                                                                    rul_prediction_rounded, 
                                                                    true_rul))

print(cScores)
#print("RMSE: {}".format(rmse))
print("RMSE2: {}".format(rmse2))
print("RHS: {}".format(rhs))
print("Time : {} seconds".format(time))

100/100 [==============================] - 0s 57us/step
scores
Engine 1, Predicted RUL 113.752197265625, Rounded RUL 113.0, Real RUL 112.0
Engine 2, Predicted RUL 118.67422485351562, Rounded RUL 118.0, Real RUL 98.0
Engine 3, Predicted RUL 64.5412368774414, Rounded RUL 64.0, Real RUL 69.0
Engine 4, Predicted RUL 86.84994506835938, Rounded RUL 86.0, Real RUL 82.0
Engine 5, Predicted RUL 99.60465240478516, Rounded RUL 99.0, Real RUL 91.0
Engine 6, Predicted RUL 101.9184341430664, Rounded RUL 101.0, Real RUL 93.0
Engine 7, Predicted RUL 105.67770385742188, Rounded RUL 105.0, Real RUL 91.0
Engine 8, Predicted RUL 87.58531188964844, Rounded RUL 87.0, Real RUL 95.0
Engine 9, Predicted RUL 114.39801025390625, Rounded RUL 114.0, Real RUL 111.0
Engine 10, Predicted RUL 89.40138244628906, Rounded RUL 89.0, Real RUL 96.0
Engine 11, Predicted RUL 94.41106414794922, Rounded RUL 94.0, Real RUL 97.0
Engine 12, Predicted RUL 102.80496978759766, Rounded RUL 102.0, Real RUL 124.0
Engine 13, Predicted RU